In [1]:
import pandas as pd
from browser import Pops, SPops

import numpy as np
from tools1kg import path_1000G
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
from scipy.ndimage import gaussian_filter1d
plt.style.use('ggplot')

In [2]:
Pops = {i:dict(zip(['pop', 'description'], Pops[i].split('--'))) for i in range(26)}

In [3]:
import sys
sys.path.append('/home/enes/pyslurm/') # https://github.com/enesdilber/pyslurm/
sys.path.append('/home/enes/bim/') # https://github.com/enesdilber/bim/
from pyslurm import Slurm
from utils import intersect_with_weights

In [4]:
Train = False

In [5]:
BIM = '/home/enes/bim/BIM.py'
locvars = ['Chromosome', 'start', 'end']
#sys.path.append(bim_path)

In [6]:
slurm = Slurm()
srun = slurm.batch('module load python3.8-anaconda/2020.07', 
                   '#mem-per-cpu=6000',
                   '#job-name="1kg"',
                   '#time=0-10:00:00',
                   '#cpus-per-task=1')

## Get statistics
Below calculates the several genome scan statistic including btree and bsfs. You should give the `Settings` file. It's a json file and will be red as a python dictionary. Its md5 hash will make sure the results are reproducible.

In [7]:
SFS = np.load("popSFS.npy", allow_pickle=True).item()
N = {i: np.array(SFS[i]).shape[1]+1 for i in range(26)}

In [9]:
if Train:
    gsettings = ['--eta=etas.json', '--r2t=0.0025', '--r2s=0.0025', '--wsz=100000', '--ssz=25000', '--log_pdf=logfr']
    jobs = {}
    for chrno in range(1, 23):
        for popid in range(26):     
            
            Ni = str(N[popid])
            popid = str(popid)
            chrno = str(chrno)
            job = ' '.join(['python', BIM, path_1000G(chrno), Ni, '--pop='+popid, '--out=raw/none'+popid+'_'+chrno+'Cmedi.csv']+gsettings)
            jobs[chrno, popid] = srun.run(job)

In [14]:
slurm.queue()

,JOBID,PARTITION,NAME,USER,ACCOUNT,ST,TIME,NODES,NODELIST(REASON)
0,19564801,standard,sys/dash,enes,stats_de,R,1-09:57:02,1,gl3122


In [8]:
if Train:
    gsettings = ['--eta=etas.json', '--r2t=0.0025', '--r2s=0.0025', '--wsz=10000', '--ssz=5000', '--log_pdf=logfr']
    jobs = {}
    for chrno in range(1, 23):
        for popid in range(26):     
            
            Ni = str(N[popid])
            popid = str(popid)
            chrno = str(chrno)
            job = ' '.join(['python', BIM, path_1000G(chrno), Ni, '--pop='+popid, '--out=raw/'+popid+'_'+chrno+'Cmedi.csv']+gsettings)
            jobs[chrno, popid] = srun.run(job)

## Merge dataframes
Above code will create 26x22 csv files for each population chromosome combination, I will merge them for each population and calculate Neutrality tests.

In [67]:
if Train:
    def read_and_load(pop, chrno):
        df2 = pd.read_csv('raw/'+str(pop)+'_'+str(chrno)+'Cmedi.csv', comment = '#')
        df2.insert(0, "Chromosome", chrno)
        return df2
    for pop in trange(26):
        pd.concat([read_and_load(pop, chrno) for chrno in range(1, 23)]).\
        to_csv('merged/s'+str(pop)+'Cmedi.csv', index = False)

In [ ]:
pd.read_csv()

## Take avarage statistic along the chromosome for each window
I will calculate a base value for each windows by calculating mean and median for each statistic

In [9]:
if Train:
    statnames = ['SS', 'FayH', 'FulD', 'ZngE', 'TajD', 'bsfs', 'FerL', 'Colless', 'btree']
    statmedis = {}
    statmeans = {}
    for stat in tqdm(statnames):
        Stat = np.zeros((576194, 26)) # number of rows and population
        for i in trange(26):
            the_path = 'merged/s'+str(i)+'Cmedi.csv'
            df = pd.read_csv(the_path)
            Stat[:,i] = df[stat]
        statmedis[stat] = np.median(Stat, 1)
        statmeans[stat] = np.mean(Stat, 1)

    dxmedi = df[['Chromosome', 'start', 'end']].copy()
    dxmean = df[['Chromosome', 'start', 'end']].copy()
    for stat in statnames:
        dxmedi['medi_'+stat] = statmedis[stat]
        dxmean['mean_'+stat] = statmeans[stat]
    dxmedi = dxmedi[dxmedi['medi_SS']>5]
    dxmean = dxmean[dxmean['mean_SS']>5]
    dxmean.to_csv('meanstats.csv', index = False)
    dxmedi.to_csv('medianstats.csv', index = False)

## Change point detection
I will calculate the change points for each statitic and for each population. I will assume a fixed number of windows for each statistic and chromosome which is $n_{cp}=$ `chromosome length/1e7`. This number will give >300 regions along the chromosome, which is roughly the number of beneficial mutations along the genome.

In [9]:
if Train:
    avglen = 1e7
    jobs = {}
    for pop in range(26):
        for chrno in range(1, 23):
            job = ' '.join(['python segmenter.py', str(pop), str(chrno),str(avglen), 'raw/', 'medi'])
            jobs[pop, chrno] = srun.run(job)

After the jobs are done, I merge them

In [12]:
if Train:
    
    for pop in trange(26):
        dfs = []
        for chrno in range(1, 23):
            path = 'raw/s'+str(pop)+'_'+str(chrno)+'medi.csv'
            dfs.append(pd.read_csv(path))

        pd.concat(dfs).to_csv('merged/s'+str(pop)+'Cmedi.csv', index = False)

## Calculate Lag for each population
Because of LD all statistics have autocorrelation. In order to calculate avaraged statistic for each region we need the variance of the avaraged statistic. To calculate this variance, we need to take account the autocovariance matrix. Here we calculate it for all statistics and for all populations

In [124]:
if Train:

    stats = ['SSCmedi', 'TajDCmedi', 'CollessCmedi', 'bsfsCmedi', 'btreeCmedi', 
             'FulDCmedi', 'ZngECmedi', 'FerLCmedi', 'FayHCmedi']

    jobs = []
    for stat in stats:
        job = 'python calc_lag.py ' + stat + ' 2000 merged Cmedi'
        jobs.append(srun.run(job))

## Empirical p-values

In [18]:
if Train:
    
    stats = ['SSCmedi', 'TajDCmedi', 'CollessCmedi', 'bsfsCmedi', 'btreeCmedi', 
             'FulDCmedi', 'ZngECmedi', 'FerLCmedi', 'FayHCmedi']

    ann = pd.read_csv('gene_result.txt', sep="\t")
    ann = ann.rename(columns = {'start_position_on_the_genomic_accession': 'start',
                                'end_position_on_the_genomic_accession': 'end'})[['chromosome', 'start', 'end', 'Symbol']]
    ann = ann[~((ann['chromosome'] == 'X') | ((ann['chromosome'] == 'Y')))]

    ann['chromosome'] = ann['chromosome'].astype('int')
    ann = ann.sort_values(['chromosome', 'start', 'end']).reset_index(drop = True)

    for pop in trange(26):
        pop = str(pop)

        df = pd.read_csv('merged/s'+pop+'Cmedi.csv')
        df['start'] = df['Chromosome']+(df['start']/1e9)
        df['end'] = df['Chromosome']+(df['end']/1e9)

        bs = df['start'].to_numpy()
        be = df['end'].to_numpy()

        cps = (ann['chromosome']+(ann['start']/1e9)).to_numpy()
        cpe = (ann['chromosome']+(ann['end']/1e9)).to_numpy()

        dx = pd.DataFrame(intersect_with_weights(bs, be, cps, cpe, df[stats].to_numpy()))
        for i, stat in enumerate(stats):
            dx['ep_'+stat] = [x[i] for x in dx['val']]
            dx['ep_'+stat] = dx['ep_'+stat].rank(pct=True)

        dx.insert(0,"Chromosome", dx['start'].astype('int'))
        dx['start'] = np.round(1e9*(dx['start']-dx['Chromosome'])).astype('int')
        dx['end'] = np.round(1e9*(dx['end']-dx['Chromosome'])).astype('int')

        df = ann.rename(columns={'chromosome':'Chromosome'})
        df = df.merge(dx, on = locvars)[locvars+['Symbol']+['ep_'+stat for stat in stats]]
        df.to_csv('gene_scores/pop/'+pop+'Cmedi.csv', index = False)

## Stats by the Region

In [126]:
genes = [{'Gene':'EDAR', 'pop':0},
         {'Gene':'CD5', 'pop':0},
         {'Gene':'LCT', 'pop':5},
         {'Gene':'SLC45A2', 'pop':5},
         {'Gene':'HERC2', 'pop':5},
         {'Gene':'SLC24A5', 'pop':5},
         {'Gene':'CD36', 'pop':10},
         {'Gene':'APOL1', 'pop':10}]

In [127]:
stats = ['btreeCmedi', 'bsfsCmedi', 'TajDCmedi', 'CollessCmedi']
signif = 0.05
pop = ''
res = []
for gene in genes:
    
    pop = gene['pop']
    cur = gene.copy()
    cur['pop'] = Pops[pop]['pop']
    
    df = pd.read_csv('gene_scores/pop/'+str(gene['pop'])+'Cmedi.csv')    
    df = df[df['Symbol'] == gene['Gene']]
    df = df[locvars+['ep_'+stat for stat in stats]]
    
    
    cur['Chromosome'] =  df['Chromosome'].iloc[0]
    cur['start'] = df['start'].iloc[0]
    cur['end'] = df['end'].iloc[-1]
    
    for stat in stats:
        if stat[:7] == 'Colless':
            cur[stat] = (df['ep_'+stat]>(1-signif)).mean()        
        else:
            cur[stat] = (df['ep_'+stat]<signif).mean()        
    
    res.append(cur)

res = pd.DataFrame(res)
res.sort_values(['pop', 'Chromosome', 'start'])

,Gene,pop,Chromosome,start,end,btreeCmedi,bsfsCmedi,TajDCmedi,CollessCmedi
2,LCT,CEU,2,135787850,135837195,1.000000,0.454545,0.636364,1.000000
3,SLC45A2,CEU,5,33944623,33984693,1.000000,0.666667,0.444444,0.888889
4,HERC2,CEU,15,28111040,28322179,0.069767,0.023256,0.069767,0.116279
5,SLC24A5,CEU,15,48120990,48142672,0.000000,0.000000,0.000000,0.000000
0,EDAR,CHB,2,108894471,108989256,0.583333,0.000000,0.166667,1.000000
1,CD5,CHB,11,61093963,61127852,0.125000,0.000000,0.000000,0.000000
6,CD36,YRI,7,80602207,80679277,0.062500,0.062500,0.187500,0.125000
7,APOL1,YRI,22,36253071,36267531,0.000000,0.000000,0.000000,0.000000


### Replication of 1000 genomes selection paper

In [128]:
st2 = [{'Gene':'EDAR', 'Chromosome':2, 'start':109_510_927, 'end':109_605_828, 'pop':0},
       {'Gene':'CD5', 'Chromosome':11,'start':60_869_867, 'end':60_896_324, 'pop':0},
       {'Gene':'LCT', 'Chromosome':2,'start':136_545_410, 'end':136_594_750, 'pop':5},
       {'Gene':'SLC45A2', 'Chromosome':5,'start':33_944_721, 'end':33_984_835, 'pop':5},
       {'Gene':'HERC2', 'Chromosome':15,'start':28_356_186, 'end':28_567_298, 'pop':5},
       {'Gene':'SLC24A5', 'Chromosome':15,'start':48_413_169, 'end':48_434_869, 'pop':5},
       {'Gene':'CD36', 'Chromosome':7,'start':79_998_891, 'end':80_308_593, 'pop':10},
       {'Gene':'APOL1', 'Chromosome':22,'start':36_649_117, 'end':36_663_576, 'pop':10}]

In [129]:
stats = ['btreeCmedi', 'bsfsCmedi', 'TajDCmedi', 'CollessCmedi']
signif = 0.05
pop = ''
res = []
for st in st2:
    
    if st['pop'] != pop:
        df = pd.read_csv('merged/s'+str(st['pop'])+'Cmedi.csv').fillna(0)
        df = df[locvars+stats]
        for stat in stats:
            if  stat[:7] == 'Colless':
                df[stat] = -df[stat]
            df['ep_'+stat] = df[stat].rank(pct=True)<signif        

    pop = st['pop']
    chrno = st['Chromosome']
    rs, re = st['start'], st['end']
    
    nrs, nre = np.round(rs, -4), np.round(re, -4)
    re = np.where(nre>re, nre, nre+1e4).astype('int')
    rs = np.where(nrs<rs, nrs, nrs-1e4).astype('int')
    
    dx = df[df['Chromosome'] == chrno].copy().reset_index(drop = True).copy()
    dx = dx[(dx['start']>=rs) & (dx['end']<=re)]
    
    for stat in stats:
        st['pop'] = Pops[int(pop)]['pop']
        st[stat] = dx['ep_'+stat].mean()
    res.append(st)
res = pd.DataFrame(res)
res.sort_values(['pop', 'Chromosome', 'start'])

,Gene,Chromosome,start,end,pop,btreeCmedi,bsfsCmedi,TajDCmedi,CollessCmedi
2,LCT,2,136545410,136594750,CEU,1.000000,0.545455,0.454545,1.000000
3,SLC45A2,5,33944721,33984835,CEU,0.888889,0.666667,0.444444,0.888889
4,HERC2,15,28356186,28567298,CEU,1.000000,0.279070,0.720930,1.000000
5,SLC24A5,15,48413169,48434869,CEU,0.800000,0.000000,0.600000,1.000000
0,EDAR,2,109510927,109605828,CHB,0.473684,0.473684,0.894737,0.263158
1,CD5,11,60869867,60896324,CHB,0.571429,0.571429,0.714286,0.571429
6,CD36,7,79998891,80308593,YRI,0.158730,0.063492,0.111111,0.206349
7,APOL1,22,36649117,36663576,YRI,0.250000,0.000000,0.000000,0.250000


## Genome-scan p-values
Now I plot the genome-scans of all statistics for each population and save the results along with the genes

In [13]:
if Train:

    stats = ['SSCmedi', 'TajDCmedi', 'CollessCmedi', 'bsfsCmedi', 'btreeCmedi', 
             'FulDCmedi', 'ZngECmedi', 'FerLCmedi', 'FayHCmedi']

    jobs = []
    for stat in stats:
        for pop in range(26):
            job = 'python pvalscan.py ' +str(pop)+' '+ stat +' Cmedi'
            jobs.append(srun.run(job))